In [1]:
import os
import json
from dotenv import load_dotenv

In [2]:
load_dotenv('../keys.env')

True

In [6]:
youtube_api_key = os.getenv("YOUTUBE_API_KEY")

from googleapiclient.discovery import build

def create_youtube_client(api_key: str):
    return build('youtube', 'v3', developerKey=api_key)

youtube_client = create_youtube_client(youtube_api_key)


In [11]:
def get_channel_videos(channel_id: str, max_results: int = None) -> list:
    """ Get list of videos from a channel using pagination to extend API's 50 results limitation"""
    videos = []
    next_page_token = None
    
    while True:
        request = youtube_client.search().list(
            part='snippet',
            channelId=channel_id,
            maxResults=50,  # Maximum allowed per request
            pageToken=next_page_token,
            type='video'
        )
        response = request.execute()
        videos.extend(response['items'])
        
        # Stop if we've reached desired number of videos
        if max_results and len(videos) >= max_results:
            return videos[:max_results]
            
        # Check if there are more pages
        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
            
    return videos

def get_video_details(video_id: str):
    request = youtube_client.videos().list(
        part='snippet,contentDetails,statistics',
        id=video_id
    )
    response = request.execute()
    return response['items'][0]

def get_channel_id(channel_name: str):
    request = youtube_client.channels().list(
        part='snippet',
        forUsername=channel_name
    )
    response = request.execute()
    return response['items'][0]['id']


In [ ]:
CHANNEL_NAME = ''

channel_id = get_channel_id(CHANNEL_NAME)
print(channel_id)

In [14]:
get_channel_videos(channel_id, 100)

1